In [1]:
import pandas as pd
import numpy as np
import pygrib as gb
import os

### Collating Forecasts

NDFD provides its data in the form of grib2 files which each contain a series of messages representing a forecast. The following code goes through all the forecasts and selects the ones closest to stranding locations

In [22]:
def calculate_distances(lat1, lon1, lat2, lon2):
    '''
    Calculates the distance between two lat,lon points using haversine formula
    :param lat1: List of latitudes of first points
    :param lon1: List of longitudes of first points
    :param lat2: List of latitudes of second points
    :param lon2: List of longitudes of second points
    :return Distance between two points
    '''
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    R = 6371 # Average radius of Earth (km)
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R * c
    return d

def find_forecast(star_id, lat, lon, df):
    '''
    Finds the location of forecasts closest to the provided lat, lon point
    :param id: STAR ID of the stranding report
    :param lat: Latitude of stranding location
    :param lon: Longitude of stranding location
    :param df: Dataframe containing forecasts from grib2 file
    :return: Pandas dataframe containing latitude, longitude point for forecast
    '''
    distances = calculate_distances(lat, lon, df['Latitude'].values, df['Longitude'].values)
    lat = df.iloc[distances.argmin()]['Latitude']
    lon = df.iloc[distances.argmin()]['Longitude']
    points = df.where((df['Latitude'] == lat) & (df['Longitude'] == lon)).dropna(how='all')
    points['STAR_ID'] = star_id
    return points.iloc[0]

def get_locations(strandings):
    '''
    Finds the closest locations of forecasts associated with all strandings
    :param strandings: Dataframe containing data for animal strandings
    :return Pandas dataframe containing lat,lon points for all forecast points near strandings
    '''
    grbs = gb.open('/Volumes/Seagate/ybq_extracted/9959_NDFD_YBQ_20070227/YBQZ97_KWBN_200702270532')
    grb = grbs[1] #Get a grib message to obtain the locations of forecasts
    lats, lons = grb.latlons()
    lats = lats.reshape(-1)
    lons = lons.reshape(-1)
    values = grb.values.reshape(-1)
    d = {'date': grb.validityDate, 'time': grb.validityTime, 'Latitude': lats, 'Longitude': lons, 'values': values}
    df = pd.DataFrame(data = d).dropna()
    df = df.round(4).reset_index()
    locations = strandings.apply(lambda row: find_forecast(row['STAR_ID'], row['Latitude'], row['Longitude'], df), axis=1)
    locations = locations.reset_index().drop(['index', 'date', 'time', 'values'], axis=1)
    return locations

In [4]:
def format_time(time):
    '''
    Formats given time to hh:mm to make conversion to pandas datetime objects easier
    :param time: Time in the format hmm
    :return Time in the format hh:mm
    '''
    while(len(time) < 4):
        time = '0' + time
    time = time[:-2] + ':' + time[-2:]
    return time

def find_values(grb, locations):
    '''
    Finds all the forecasts in a grib message associated with provided locations
    :param grb: grib file message
    :param locations: Dataframe containing desired forecast locations (lat, lon)
    :return Forecasts found in grib message at provided points
    '''
    lats, lons = grb.latlons()
    lats = lats.reshape(-1).round(4)
    lons = lons.reshape(-1).round(4)
    values = grb.values.reshape(-1)

    idxs = np.in1d(lats, locations['Latitude'])
    idxs = np.where(idxs)[0]
    real_values = np.nonzero(values[idxs].mask)[0]
    idxs = np.delete(idxs,real_values)

    validity_time = format_time(str(grb.validityTime))
    data_time = format_time(str(grb.dataTime))
    validity_datetime = str(grb.validityDate) + ' ' + str(validity_time)
    
    data_datetime = str(grb.dataDate) + ' ' + str(data_time)

    validity_datetimes = np.repeat(validity_datetime, len(values[idxs]))
    data_datetimes = np.repeat(data_datetime, len(values[idxs]))
    d = {'Validity_Date': validity_datetimes, 'Data_Date': data_datetimes, 'Latitude': lats[idxs], 
         'Longitude': lons[idxs], 'Value': values[idxs]}
    df = pd.DataFrame(d)
    df = pd.merge(left=df,right=locations, on=['Latitude', 'Longitude'], how='inner')
    return df

def find_all_values(grbs, locations):
    '''
    Finds all the forecasts in a grib file at provided locations
    :param grbs: Pygrib object
    :param locations: Dataframe containing desired forecast locations (lat, lon)
    :return All forecasts in grib file found at provided points
    '''
    values = pd.DataFrame(columns=['Validity_Date', 'Data_Date', 'Latitude', 'Longitude', 'Value', 'STAR_ID'])
    for grb in grbs:
        v = find_values(grb, locations) 
        values = pd.concat([values,pd.DataFrame(v)])
    return values

In [ ]:
%%time

results = []
path = []
files = []
pathfiles = []

strandings = pd.read_excel('/Users/christopherberglund/Projects/stranding/ES Strandings 2016-2018.xlsx')
strandings = strandings.set_index('STAR_ID').dropna(subset=['Animal::TMMC_ID'])
strandings = strandings.where(strandings['County'].isin(['Monterey', 'Santa Cruz', 'San Mateo'])).dropna(how='all')
strandings = pd.DataFrame(strandings[['Latitude', 'Longitude']]).round(4)

locations = get_locations(strandings.reset_index())

for (dirpath, dirnames, filenames) in os.walk('/Volumes/Seagate/ybq_extracted'):
    path.append(dirpath)
    files.append(filenames)
    
for path, f in zip(path, files):
    for file in f:
        pathfiles.append(path+'/'+file)
        
df = pd.DataFrame(columns=['Validity_Date', 'Data_Date', 'Latitude', 'Longitude', 'wvht'])


for file in pathfiles:
    grbs = gb.open(file)
    v = find_all_values(grbs, locations)
    results.append(v)
df = pd.concat(results)

df.to_csv('/Volumes/Seagate/wdir.csv')
print('finished')

/Users/christopherberglund/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [88]:
ndfd_data = pd.read_csv('/Users/christopherberglund/Projects/stranding/ndfd_total_wdir.csv')
ndfd_data['Validity_Date'] = pd.to_datetime(ndfd_data['Validity_Date'])
ndfd_data['Data_Date'] = pd.to_datetime(ndfd_data['Data_Date'])
ndfd_data['dData_Date'] = (ndfd_data['Validity_Date'] - ndfd_data['Data_Date']).astype('timedelta64[h]')
ddata_date_min = ndfd_data.groupby(['Latitude', 'Longitude'])['dData_Date'].transform(min)
ndfd_data = ndfd_data.loc[ndfd_data['dData_Date'] == ddata_date_min]
ndfd_data.to_csv('/Users/christopherberglund/Projects/stranding/ndfd_local_wdir.csv', index=False)